In [3]:
import pandas as pd
import h5py
import os
from PIL import Image
import numpy as np

# set path
csv_file_path = r'C:\Users\dengz\Desktop\DeepLearninig7318\A2\histopathologic-cancer-detection\train_labels.csv'  # Replace 'your_labels_file.csv' with the name of your CSV file
image_folder = r'C:\Users\dengz\Desktop\DeepLearninig7318\A2\histopathologic-cancer-detection\train'
hdf5_file_path = r'C:\Users\dengz\Desktop\DeepLearninig7318\A2\histopathologic-cancer-detection\matched_images.h5'

# load csv
labels_df = pd.read_csv(csv_file_path)

# create HDF5
with h5py.File(hdf5_file_path, 'w') as hf:
    # images and labels
    num_images = len(labels_df)
    sample_image_path = os.path.join(image_folder, labels_df.iloc[0]['id'] + '.tif')  # assuming the extension is .tif
    image_shape = np.array(Image.open(sample_image_path)).shape
    images_dset = hf.create_dataset("images", shape=(num_images, *image_shape), dtype=np.uint8)
    labels_dset = hf.create_dataset("labels", shape=(num_images,), dtype=np.uint8)
    
    for idx, row in labels_df.iterrows():
        image_path = os.path.join(image_folder, row['id'] + '.tif')  # assuming the extension is .tif
        image_data = np.array(Image.open(image_path))
        
        images_dset[idx] = image_data
        labels_dset[idx] = row['label']


In [4]:
import h5py
import numpy as np
from sklearn.model_selection import train_test_split

data_path = r'C:\Users\dengz\Desktop\DeepLearninig7318\A2\histopathologic-cancer-detection\matched_images.h5'

with h5py.File(data_path, 'r') as hf:
    images = hf['images'][:]
    labels = hf['labels'][:]

# Use stratified sampling to divide the dataset on a 90-5-5 scal
train_images, temp_images, train_labels, temp_labels = train_test_split(
    images, labels, test_size=0.1, random_state=42, stratify=labels
)
val_images, test_images, val_labels, test_labels = train_test_split(
    temp_images, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)

output_hdf5_path = r'C:\Users\dengz\Desktop\DeepLearninig7318\A2\histopathologic-cancer-detection\split_data.h5'

with h5py.File(output_hdf5_path, 'w') as hf:

    hf.create_dataset("train_images", data=train_images, dtype=np.uint8)
    hf.create_dataset("train_labels", data=train_labels, dtype=np.uint8)

    hf.create_dataset("val_images", data=val_images, dtype=np.uint8)
    hf.create_dataset("val_labels", data=val_labels, dtype=np.uint8)

    hf.create_dataset("test_images", data=test_images, dtype=np.uint8)
    hf.create_dataset("test_labels", data=test_labels, dtype=np.uint8)


In [3]:
import h5py
import numpy as np

# 函数：裁剪图像的中心32x32区域
def crop_center(img):
    y, x, _ = img.shape
    start_x = x//2 - 16
    start_y = y//2 - 16
    return img[start_y:start_y+32, start_x:start_x+32]

# 从split_data.h5加载数据
with h5py.File('split_data.h5', 'r') as hf:
    train_images = hf['train_images'][:]
    val_images = hf['val_images'][:]
    test_images = hf['test_images'][:]
    train_labels = hf['train_labels'][:]
    val_labels = hf['val_labels'][:]
    test_labels = hf['test_labels'][:]

# 裁剪图像
cropped_train_images = np.array([crop_center(img) for img in train_images])
cropped_val_images = np.array([crop_center(img) for img in val_images])
cropped_test_images = np.array([crop_center(img) for img in test_images])

# 保存裁剪后的数据到新的 HDF5 文件
cropped_hdf5_path = r'C:\Users\dengz\Desktop\DeepLearninig7318\A2\histopathologic-cancer-detection\cropped_data.h5'
with h5py.File(cropped_hdf5_path, 'w') as hf:
    hf.create_dataset("train_images", data=cropped_train_images, dtype=np.uint8)
    hf.create_dataset("train_labels", data=train_labels, dtype=np.uint8)
    hf.create_dataset("val_images", data=cropped_val_images, dtype=np.uint8)
    hf.create_dataset("val_labels", data=val_labels, dtype=np.uint8)
    hf.create_dataset("test_images", data=cropped_test_images, dtype=np.uint8)
    hf.create_dataset("test_labels", data=test_labels, dtype=np.uint8)


In [1]:
import h5py
import numpy as np

# Crop the centre 32x32 area of the image
def crop_center(img):
    y, x, _ = img.shape
    start_x = x//2 - 16
    start_y = y//2 - 16
    return img[start_y:start_y+32, start_x:start_x+32]

# Crop training, validation and test images
def crop_images(images):
    return np.array([crop_center(img) for img in images])

with h5py.File('split_data.h5', 'r') as hf:
    train_images = hf['train_images'][:]
    val_images = hf['val_images'][:]
    test_images = hf['test_images'][:]

cropped_train_images = crop_images(train_images)
cropped_val_images = crop_images(val_images)
cropped_test_images = crop_images(test_images)

output_path = r'C:\Users\dengz\Desktop\DeepLearninig7318\A2\histopathologic-cancer-detection\cropped_data.h5'
with h5py.File(output_path, 'w') as hf:
    
    hf.create_dataset("train_images", data=cropped_train_images, dtype=np.uint8)
    hf.create_dataset("val_images", data=cropped_val_images, dtype=np.uint8)
    hf.create_dataset("test_images", data=cropped_test_images, dtype=np.uint8)

    hf.create_dataset("train_labels", data=hf['train_labels'][:], dtype=np.uint8)
    hf.create_dataset("val_labels", data=hf['val_labels'][:], dtype=np.uint8)
    hf.create_dataset("test_labels", data=hf['test_labels'][:], dtype=np.uint8)


KeyError: "Unable to open object (object 'train_labels' doesn't exist)"